In [4]:
import re
from typing import List, Dict, Tuple, Optional

def _split_entries_on_top_level_semicolons(text: str) -> List[str]:
    """
    Split Windisch page text into entries on semicolons that are NOT inside parentheses.
    Keeps all punctuation/spacing; trims outer whitespace on each chunk.
    """
    parts, buf, depth = [], [], 0
    for ch in text:
        if ch == '(':
            depth += 1
        elif ch == ')':
            depth = max(depth - 1, 0)
        if ch == ';' and depth == 0:
            parts.append(''.join(buf).strip())
            buf = []
        else:
            buf.append(ch)
    # last chunk (possibly open-ended if the page cuts mid-entry)
    last = ''.join(buf).strip()
    if last:
        parts.append(last)
    # drop empty fragments quietly
    return [p for p in parts if p]

def extract_raw_entries(volume: str, page: str, page_text: str) -> List[Dict[str, str]]:
    """
    Minimal extractor: returns entries as raw strings with location.
    - volume: e.g. "I" or "II" (string so you can use roman numerals as-is)
    - page:   page label as a string (e.g. "4")
    - page_text: the exact text of that page (one big string)

    Returns: [{"volume": ..., "page": ..., "raw": ...}, ...]
    """
    # normalize trivial page artifacts (keep the content 'raw' otherwise)
    cleaned = page_text.strip()
    # split on top-level semicolons
    chunks = _split_entries_on_top_level_semicolons(cleaned)
    # keep only chunks that look like actual entries (start with two single-quotes)
    # but don't be too strict — allow leading whitespace
    entry_like = []
    for c in chunks:
        if re.search(r"\b''[^']+", c):  # has at least one quoted head like ''ai aiə''
            entry_like.append(c.strip())
        else:
            # if you want literally everything (even trailing editorial notes), keep this:
            entry_like.append(c.strip())

    return [{"volume": volume, "page": page, "raw": c} for c in entry_like]

# --- optional convenience wrapper if you keep a page store in a dict ---
def extract_raw_entries_from_pages(volume: str, page: str,
                                   pages: Dict[Tuple[str, str], str]) -> List[Dict[str, str]]:
    """
    Same as extract_raw_entries, but looks up the page text in a {(volume, page): text} dict.
    """
    key = (volume, page)
    if key not in pages:
        raise KeyError(f"No page text found for volume={volume}, page={page}")
    return extract_raw_entries(volume, page, pages[key])


# -------------------------
# Example usage (with your p.4 section)
# -------------------------
if __name__ == "__main__":
    PAGES = {
        ("I", "4"): """''ai aiə'' „gesicht“, air. aged; ''aiəl̄ ail̄'' „hitze“ (vgl. II 251, 14), air. adall?; ''aiən'' „kessel“, air. aigen; ''aiərk airk'' (vgl. II 251, 15) „horn, geweih“, air. adarc; ''auən̄ aun̄'' (vgl. II 251, 15) „fluss“, air. abann; ''ā'' „glück, darrofen, furt“, mir. ág, mir. áith, air. áth; ''āwul āwl̥'' „glücklich“, ághamhal (Molloy 49: áthúil); ''bau'' „bogen“, mir. boga, aengl. boga; ''bauər baur'' (vgl. II 251, 16) „taub“, air. bodar; ''bā'' „zuneigung“, mir. báid báde (Molloy 35: báighe); ''bā'' „ertränken, baden“ (Molloy 81: bághthadh), air. bádud; ''bāĭm'' „ertränke, mache nass“, mir. báidim; ''bēl'' „mund“, air. bél; ''bēl'' (neben ''biəl'') „beil“, air. biail; ''bēs bēsə'' (vgl. II 262, 26 und Molloy 33) „sitte, betragen, gewohnheit“, air. bés; ''bǡ'' „nahrung“ (vgl. II 263, 15), beathughadh, Keat., ''bĭaiəx'' „tier“, beathadhach, Keat., beide von beatha, air. bethu; ''bĭai bai'', 3. sing. fut. zu ''tāĭm'' „ich bin“, air. biaid bieid; ''bĭō'' „lebendig“ air. béo bíu; ''blā'' „blüte, blume“, mir. bláth; ''blāklī́ blāḱlī́'' „Dublin“, Baile-atha-cliath (die erste silbe als schwach betonte ist eingipflig); ''blāx'' „buttermilch“, mir. bláthach; ''blā'' „melken“, wohl neugebildete infinitivform zu bleaghaim, O’R., statt blighim nach bleaghan aus mir. blegon, dabei unter einfluss von ''blāx'' „buttermilch“ (''blān'' „melken“, mir. blegon, scheint eingipflig zu sein); ''blīm'' „melke“, mir. bligim; ''bō'' „kuh“, air. bó; ''bōhr̥'' „weg“, mir. bóthar; ''bōhŕīn'' „feldweg, gasse“, von ''bōhr̥''; ''brāhŕ̥'' „klosterbruder“, air. bráthir; ''brāx'' „jüngstes gericht“, air. bráth; ''brōg'' „schuh“, mir. bróc, aengl. bróc, anord. brókr; ''bŕǡ'' „schön“, mir. breagha (O’Clery); ''bŕǡxə bŕǡxcə'' „schönheit“, von ''bŕǡ''; ''bŕēg'' „lüge“, air. bréc; ''bŕēgān'' „spielzeug“, von ''bŕēg''; ''bŕēgəx'' „lügnerisch“, mir. brécach; ''bŕīȷ'' „Brigitta“, air. Brigit; ''bŕō'' „bedrücken“, Keat. breódh; ''bŕōĭm'' „bedrücke“, Keat. breódhaim, breóghaim; ''dauəx daux'', f., „fass“ (vgl. II 251, 15. 266, 5), aschott. dabach (Bk. of Deer); ''dauən daun'' „welt“ (vgl. II 251, 16), air. domun; ''dauən daun'' „tiefe“, air. domain; ''dəlī́ ȷlī'' (neben ''dəlíə'') „gesetz“ (vgl. II 251, 17), air. dliged; ''dəlū''? „kette, werfte“ (des gewebes) (vgl. II 266, 23), air. dlúth; ''dō'' „brennen“, Molloy 81: dóghadh, ''dōĭm'' „brenne, verbrenne“, dóghaim dóighim, mir. dóud dóim; ''ȷauəl ȷaul'' „teufel“, air. diabul; ''ȷēgə'' „göttlich“, diaga (Molloy 50) (Keat. diadha aus air. diade); ''ȷēŕḱə'' (nicht ''ȷēŕḱ'', wie II 80, 32) „almosen“, déirc (Molloy 35: déirce) mir.<noinclude>"""
    }

    entries = extract_raw_entries_from_pages("I", "4", PAGES)
    for i, e in enumerate(entries, 1):
        print(f"{i:02d}. [{e['volume']}:{e['page']}] {e['raw']}\n")


01. [I:4] ''ai aiə'' „gesicht“, air. aged

02. [I:4] ''aiəl̄ ail̄'' „hitze“ (vgl. II 251, 14), air. adall?

03. [I:4] ''aiən'' „kessel“, air. aigen

04. [I:4] ''aiərk airk'' (vgl. II 251, 15) „horn, geweih“, air. adarc

05. [I:4] ''auən̄ aun̄'' (vgl. II 251, 15) „fluss“, air. abann

06. [I:4] ''ā'' „glück, darrofen, furt“, mir. ág, mir. áith, air. áth

07. [I:4] ''āwul āwl̥'' „glücklich“, ághamhal (Molloy 49: áthúil)

08. [I:4] ''bau'' „bogen“, mir. boga, aengl. boga

09. [I:4] ''bauər baur'' (vgl. II 251, 16) „taub“, air. bodar

10. [I:4] ''bā'' „zuneigung“, mir. báid báde (Molloy 35: báighe)

11. [I:4] ''bā'' „ertränken, baden“ (Molloy 81: bághthadh), air. bádud

12. [I:4] ''bāĭm'' „ertränke, mache nass“, mir. báidim

13. [I:4] ''bēl'' „mund“, air. bél

14. [I:4] ''bēl'' (neben ''biəl'') „beil“, air. biail

15. [I:4] ''bēs bēsə'' (vgl. II 262, 26 und Molloy 33) „sitte, betragen, gewohnheit“, air. bés

16. [I:4] ''bǡ'' „nahrung“ (vgl. II 263, 15), beathughadh, Keat., ''bĭaiəx'' „tier“